# Usando Web APIs

Una API o *Application Program Interface* permite que uno pueda "hablar" con algún programa. Muchos sitios web o servicios proveen una API para poder consultar información de manera automatizada.

Para mapeo y análisis espacial, ser capaz de usar APIs es crítico. Por mucho tiempo, la API de Google Maps fué la más popular de la Web. Las APIs permiten que puedas consultar a los servidores web y obtener resultados sin bajar datos o realizar cómputos en tu computadora.

Los casos de uso mas comunes para las APIs en análisis espacial son:
- Obtener direcciones / ruteo
- Optimización de rutas
- Geocodificación
- Bajar datos
- Obtener datos en tiempo real
- ...

Los proveedores de tales APIs disponen de muchas maneras de implementarlas. Existen estándares tales como REST, SOAP, GraphQL, y otros. *REST* es el más popular en la web, y para las aplicaciones geoespaciales. Las APIs REST se utilizan sobre el protocolo HTTP y por tanto también se las llaman APIs web.

## Entendiendo JSON y GeoJSON

JSON es el acrónimo de JavaScript Object Notation. Es un formato para almacenar y transportar datos y es el estándar de facto para intercambio de datos en APIs. GeoJSON es una extensión de JSON que se utiliza para representar datos espaciales.

Python incluye un módulo `json` que posee métodos para leer datos json y convertirlos en objetos Python y viceversa. En este ejemplo, usaremos el módulo `requests` para consultar los datos a la API,
Python has a built-in `json` module that has methods for reading json data and converting it to Python objects, and vice-versa. In this example, we are using the `requests` module for querying the API which conveniently does the conversion for us. But it is useful to learn the basics of working with JSON in Python.

The GeoJSON data contains *features*, where each feature has some *properties* and a *geometry*.

In [ ]:
geojson_string = '''
{
  "type": "FeatureCollection",
  "features": [
    {"type": "Feature",
      "properties": {"name": "San Francisco"},
      "geometry": {"type": "Point", "coordinates": [-121.5687, 37.7739]}
    }
  ]
}
'''
print(geojson_string)

Para convertir una cadena JSON a objeto Python usamos el método `json.loads()`.

In [ ]:
import json

data = json.loads(geojson_string)
print(type(data))
print(data)

Ahora que hemos *parseado* la cadena de GeoJson y tenemos un objeto, podemos extraer información de él. Los datos están almacenados en una `FeatureCollection`, que es una lista de `features`.

Now that we have *parsed* the GeoJSON string and have a Python object, we can extract infromation from it. The data is stored in a *FeatureCollection* - which is a list of *features*.

En nuestro ejemplo, sólamente tenemos una feature, así que podemos accederla con el índice `0`.

In [ ]:
city_data = data['features'][0]
print(city_data)

La representación de una feature es un diccionario, por tanto los ítems individuales se acceden por sus claves.

In [ ]:
city_name = city_data['properties']['name']
city_coordinates = city_data['geometry']['coordinates']
print(city_name, city_coordinates)

## El módulo `requests`

Para consultar un servidor, lo más probable es que enviemos una consulta *GET* con algunos parámetros, y el servidor nos contesta una respuesta en algún formato, usualmente JSON. El módulo `requests` permite enviar consultas y *parsear* las respuestas a través de Python.

La respuesta contiene los datos recibidos del servidor. Contiene un [*código de status* HTTP](https://developer.mozilla.org/es/docs/Web/HTTP/Status) que nos indica si el pedido fué exitoso. El código "200" significa que todo salió OK.


In [5]:
import requests

response = requests.get('https://www.spatialthoughts.com')

print(response.status_code)

200


## Usando la API de OpenSky Network

OpenSky Network es un servicio que permite obtener información de todos las aeronaves que están en vuelo a nivel global. Esta información está accesible [a través de una API](https://openskynetwork.github.io/opensky-api/index.html).
Vamos a utilizar esta API para generar un archivo CSV con la información de las aeronaves que estén en determinado sector del globo.

Primero importamos las bibliotecas que necesitaremos.

In [30]:
import requests
import json
import csv
from pathlib import Path

Luego indicamos dónde guardaremos los datos que bajaremos. Vamos a generar un archivo en el directorio en el que está este notebook.

In [31]:
# Tomo el directorio donde se ubica el notebook
DIRECTORY = Path('.')

# Archivo donde quedan los datos en CSV
CSV_FILENAME = DIRECTORY / Path("datos.csv")

# Si el nombre existe y es un archivo no es problema, porque es una versión anterior y la sobreescribiremos con datos nuevos
if CSV_FILENAME.is_dir():
    raise Exception("El nombre ya existe y es un directorio")
    

Definimos área usando coordenadas. En este caso un área que abarque a sudamérica.

In [32]:
lon_max, lat_max = -33.4, 13.0
lon_min, lat_min = -81.0, -56.0

Generamos la URL para comunicarnos con la API. 

In [33]:
# Construcción de la query para la API REST

# generamos la parte de parámetros, usamos sólamente las coordenadas
params = urlencode(
    {"lamin": lat_min, "lomin": lon_min, "lamax": lat_max, "lomax": lon_max}
)
# generamos las partes de la url
neturl = "https://opensky-network.org/api/states/all"

# ensamblamos todas las partes
DATA_URL = neturl + "?" + params

Definimos los nombres de las columnas que va a tener el archivo generado.

In [21]:
# Nombres de las columnas
# ver https://openskynetwork.github.io/opensky-api/rest.html#response
COL_NAMES = (
    "icao24 callsign origin_country time_position last_contact long "
    + "lat baro_altitude on_ground velocity true_track vertical_rate "
    + "sensors geo_altitude squawk spi position_source category"
).split()

Acá es donde comenzamos a juntar todo. Vamos a definir tres funciones. Una para obtener los datos, otra para darles forma y la final para escribir todo en un archivo.

In [34]:
def get_data_as_json(url):
    """
    Obtengo datos de la web de OpenSky
    'url' contiene la query REST con los parámetros adecuados
    Retorna un objeto con los datos recibidos
    """

    req = requests.get(url)
    result = req.status_code == 200 and req.json() or None
    return result


def format_data(data):
    """
    Genero una lista de datos con encabezados.
    Cada elemento de la lista es una línea (registro).
    'data' proviene del json generado por la API de OpenSky
    """

    return [COL_NAMES] + data["states"]


def save_data_as_csv(data, filename):
    """
    Guarda datos en formato CSV
    'data' es una lista
    'filename' es un nombre de archivo válido
    """
    # https://docs.python.org/3/library/csv.html#csv.writer
    with open(filename, "w") as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerows(data)

Finalmente una función para hacer todo el trabajo con una sola llamada.

In [25]:
# Obtiene los datos de una URL y los guarda en formato CSV en el archivo
# indicado en filename
def do_it(url, filename):
    try:
        data = get_data_as_json(url)
        data = format_data(data)
        save_data_as_csv(data, filename)
        print("Todo OK. Archivo generado")
    except:
        print(e)


In [28]:
do_it(DATA_URL, CSV_FILENAME)

Todo OK. Archivo generado


### Ejercicios

- (Fácil) Cambiar las coordenadas del área de sobre la que se filtrarán los datos.
- (Menos fácil) Filtrar también las aeronaves por país de origen
- (No tan fácil) Realizar los cambios necesarios para un mejor control de errores. Contemplar qué pasa si
    * hay un problema en la red y no se reciben datos
    * no se puede escribir el archivo
    Imprimir mensajes claros que indiquen qué sucedió.